In [1]:
# ปรับแต่งค่าการทำงานของ Spark
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("KafkaSubscribe").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1000m").\
        config("spark.executor.cores", "2").\
        config("spark.cores.max", "6").\
        config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0").\
        getOrCreate()



Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-81ce6043-3a2c-4650-9c8e-23026639db7b;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 607ms :: artifacts dl 14

In [2]:
rawMetadata_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "10.128.0.12:9092") \
  .option("subscribe", "quickstart-events") \
  .option("group.id", "Aekanun-Spark-App") \
  .load()

In [3]:
from pyspark.sql import functions as sparkf

In [4]:
rawMetadata_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
# เลือกเฉพาะคอลัมน์ที่ต้องการ
onlyMetadata_df = rawMetadata_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [6]:
from pyspark.sql.types import *
from pyspark.sql.functions import from_json, col

# Main schema
schema = StructType([
    StructField("locationId", StringType()),
    StructField("location", StringType()),
    StructField("parameter", StringType()),
    StructField("value", StringType()),
    StructField("date", StringType()),
    StructField("unit", StringType()),
    StructField("coordinates", StringType()),
    StructField("country", StringType()),
    StructField("city", StringType()),
    StructField("isMobile", StringType()),
    StructField("isAnalysis", StringType()),
    StructField("entity", StringType()),
    StructField("sensorType", StringType())
])

# Nested 'date' and 'coordinates' schema
date_schema = StructType([
    StructField("utc", StringType()),
    StructField("local", StringType())
])

coordinates_schema = StructType([
    StructField("latitude", DoubleType()),
    StructField("longitude", DoubleType())
])

# Parse the JSON string column and convert it to a struct.
parsedData_df = onlyMetadata_df.withColumn("data", from_json("value", schema))

In [7]:
parsedData_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- locationId: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- parameter: string (nullable = true)
 |    |-- value: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- unit: string (nullable = true)
 |    |-- coordinates: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- isMobile: string (nullable = true)
 |    |-- isAnalysis: string (nullable = true)
 |    |-- entity: string (nullable = true)
 |    |-- sensorType: string (nullable = true)



In [8]:
# Select only the nested fields
unNested_df = parsedData_df.select("key", "data.*")

In [9]:
unNested_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- date: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)



In [10]:
# Parse the JSON string column and convert it to a struct.
unNested_df.withColumn("data", from_json("value", schema))\
.withColumn("date", from_json("date", date_schema))\
.withColumn("coordinates", from_json("coordinates", coordinates_schema))\
.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- date: struct (nullable = true)
 |    |-- utc: string (nullable = true)
 |    |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- locationId: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- parameter: string (nullable = true)
 |    |-- value: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- unit: st

In [11]:
extractedDateLatLong_df = unNested_df.withColumn("data", from_json("value", schema))\
.withColumn("date", from_json("date", date_schema))\
.withColumn("coordinates", from_json("coordinates", coordinates_schema))\
.select('key',
 'locationId',
 'location',
 'parameter',
 'value',
 'date.*',
 'unit',
 'coordinates.*',
 'country',
 'city',
 'isMobile',
 'isAnalysis',
 'entity',
 'sensorType')

In [12]:
extractedDateLatLong_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- utc: string (nullable = true)
 |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)



In [13]:
final_df = extractedDateLatLong_df\
.withColumn('sourceUnixSTP',sparkf.to_timestamp(sparkf.col('utc'), "yyyy-MM-dd'T'HH:mm:ssXXX"))

In [14]:
final_df.columns

['key',
 'locationId',
 'location',
 'parameter',
 'value',
 'utc',
 'local',
 'unit',
 'latitude',
 'longitude',
 'country',
 'city',
 'isMobile',
 'isAnalysis',
 'entity',
 'sensorType',
 'sourceUnixSTP']

In [15]:
final_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- utc: string (nullable = true)
 |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)
 |-- sourceUnixSTP: timestamp (nullable = true)



In [16]:
#agg_df = final_df.groupBy('value').agg(sparkf.max(sparkf.col('value')))

In [17]:
from pyspark.sql import functions as F

In [ ]:
from pyspark.sql.functions import window

windowedCounts = final_df.groupBy(
    window(final_df.sourceUnixSTP, "3600 minutes", "3600 minutes"),
    final_df.value
).count()

query = windowedCounts.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()


23/06/19 15:32:54 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-8f7fb605-8335-4003-90d0-ab53a1680e77. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+-----+-----+
|window|value|count|
+------+-----+-----+
+------+-----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-03-31 12:00...| 23.0|    1|
|[2022-03-31 12:00...| 29.0|    1|
|[2022-03-29 00:00...| 19.0|   14|
|[2022-04-03 00:00...| 15.0|    3|
|[2022-03-29 00:00...| 16.0|   12|
|[2022-03-31 12:00...| 16.0|    1|
|[2022-03-31 12:00...| 32.0|    1|
|[2022-03-31 12:00...| 34.0|    1|
|[2022-03-29 00:00...| 15.0|    5|
|[2022-03-31 12:00...| 35.0|    2|
|[2022-03-29 00:00...| 22.0|    1|
|[2022-04-03 00:00...| 17.0|    1|
|[2022-03-31 12:00...| 24.0|    2|
|[2022-03-31 12:00...| 14.0|    3|
|[2022-03-31 12:00...| 28.0|    1|
|[2022-04-03 00:00...| 12.0|    3|
|[2022-03-29 00:00...| 21.0|    3|
|[2022-04-03 00:00...| 13.0|    9|
|[2022-03-31 12:00...| 27.0|    1|
|[2022-03-31 12:00...| 25.0|    2|
+--------------------+-----+-----+
only showing top 20 rows



-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-03-31 12:00...| 23.0|    1|
|[2022-03-31 12:00...| 29.0|    1|
|[2022-03-29 00:00...| 19.0|   14|
|[2022-04-05 12:00...| 32.0|    1|
|[2022-04-05 12:00...| 49.0|    1|
|[2022-04-03 00:00...| 15.0|    3|
|[2022-04-03 00:00...| 31.0|    3|
|[2022-04-03 00:00...| 24.0|    4|
|[2022-04-03 00:00...| 21.0|    1|
|[2022-03-29 00:00...| 16.0|   12|
|[2022-03-31 12:00...| 16.0|    1|
|[2022-04-05 12:00...| 38.0|    1|
|[2022-03-31 12:00...| 32.0|    1|
|[2022-04-03 00:00...| 25.0|    5|
|[2022-03-31 12:00...| 34.0|    1|
|[2022-03-29 00:00...| 15.0|    5|
|[2022-04-05 12:00...| 34.0|    2|
|[2022-03-29 00:00...| 22.0|    1|
|[2022-03-31 12:00...| 35.0|    2|
|[2022-04-03 00:00...| 17.0|    2|
+--------------------+-----+-----+
only showing top 20 rows



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-03-31 12:00...| 23.0|    1|
|[2022-03-31 12:00...| 29.0|    1|
|[2022-03-29 00:00...| 19.0|   14|
|[2022-04-05 12:00...| 32.0|    1|
|[2022-04-05 12:00...| 49.0|    2|
|[2022-04-05 12:00...| 56.0|    4|
|[2022-04-05 12:00...| 50.0|    7|
|[2022-04-03 00:00...| 15.0|    3|
|[2022-04-03 00:00...| 31.0|    3|
|[2022-04-05 12:00...| 41.0|    1|
|[2022-04-03 00:00...| 24.0|    4|
|[2022-04-03 00:00...| 21.0|    1|
|[2022-04-05 12:00...| 54.0|    2|
|[2022-03-29 00:00...| 16.0|   12|
|[2022-03-31 12:00...| 16.0|    1|
|[2022-04-05 12:00...| 38.0|    4|
|[2022-03-31 12:00...| 32.0|    1|
|[2022-04-08 00:00...| 51.0|    1|
|[2022-04-03 00:00...| 25.0|    5|
|[2022-03-31 12:00...| 34.0|    1|
+--------------------+-----+-----+
only showing top 20 rows



...

-------------------------------------------
Batch: 66
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-04-28 00:00...| 24.0|    7|
|[2022-03-31 12:00...| 23.0|    1|
|[2022-04-08 00:00...| 64.0|    2|
|[2022-05-10 12:00...| 12.0|   12|
|[2022-06-02 00:00...| 25.0|    2|
|[2022-03-31 12:00...| 29.0|    1|
|[2022-04-10 12:00...| 63.0|    1|
|[2022-04-18 00:00...| 50.0|    1|
|[2022-05-15 12:00...| 20.0|    3|
|[2022-06-04 12:00...| 16.0|    1|
|[2022-04-20 12:00...| 47.0|    1|
|[2022-04-30 12:00...| 38.0|    1|
|[2022-03-29 00:00...| 19.0|   14|
|[2022-05-28 00:00...| 26.0|    5|
|[2022-04-05 12:00...| 32.0|    1|
|[2022-06-07 00:00...| 16.0|   19|
|[2022-05-08 00:00...| 24.0|    9|
|[2022-04-30 12:00...| 24.0|    7|
|[2022-04-05 12:00...| 49.0|    2|
|[2022-04-08 00:00...| 55.0|    2|
+--------------------+-----+-----+
only showing top 20 rows



-------------------------------------------
Batch: 67
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-06-09 12:00...| 12.0|    3|
|[2022-04-28 00:00...| 24.0|    7|
|[2022-03-31 12:00...| 23.0|    1|
|[2022-04-08 00:00...| 64.0|    2|
|[2022-05-10 12:00...| 12.0|   12|
|[2022-06-02 00:00...| 25.0|    2|
|[2022-03-31 12:00...| 29.0|    1|
|[2022-04-10 12:00...| 63.0|    1|
|[2022-04-18 00:00...| 50.0|    1|
|[2022-05-15 12:00...| 20.0|    3|
|[2022-06-04 12:00...| 16.0|    1|
|[2022-04-20 12:00...| 47.0|    1|
|[2022-04-30 12:00...| 38.0|    1|
|[2022-03-29 00:00...| 19.0|   14|
|[2022-05-28 00:00...| 26.0|    5|
|[2022-04-05 12:00...| 32.0|    1|
|[2022-06-07 00:00...| 16.0|   19|
|[2022-05-08 00:00...| 24.0|    9|
|[2022-04-30 12:00...| 24.0|    7|
|[2022-04-05 12:00...| 49.0|    2|
+--------------------+-----+-----+
only showing top 20 rows



-------------------------------------------
Batch: 68
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-06-09 12:00...| 12.0|    3|
|[2022-04-28 00:00...| 24.0|    7|
|[2022-03-31 12:00...| 23.0|    1|
|[2022-04-08 00:00...| 64.0|    2|
|[2022-05-10 12:00...| 12.0|   12|
|[2022-06-02 00:00...| 25.0|    2|
|[2022-03-31 12:00...| 29.0|    1|
|[2022-04-10 12:00...| 63.0|    1|
|[2022-04-18 00:00...| 50.0|    1|
|[2022-05-15 12:00...| 20.0|    3|
|[2022-06-04 12:00...| 16.0|    1|
|[2022-04-20 12:00...| 47.0|    1|
|[2022-04-30 12:00...| 38.0|    1|
|[2022-03-29 00:00...| 19.0|   14|
|[2022-05-28 00:00...| 26.0|    5|
|[2022-04-05 12:00...| 32.0|    1|
|[2022-06-07 00:00...| 16.0|   19|
|[2022-05-08 00:00...| 24.0|    9|
|[2022-04-30 12:00...| 24.0|    7|
|[2022-04-05 12:00...| 49.0|    2|
+--------------------+-----+-----+
only showing top 20 rows



-------------------------------------------
Batch: 69
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-06-09 12:00...| 12.0|    3|
|[2022-04-28 00:00...| 24.0|    7|
|[2022-03-31 12:00...| 23.0|    1|
|[2022-04-08 00:00...| 64.0|    2|
|[2022-05-10 12:00...| 12.0|   12|
|[2022-06-02 00:00...| 25.0|    2|
|[2022-03-31 12:00...| 29.0|    1|
|[2022-04-10 12:00...| 63.0|    1|
|[2022-04-18 00:00...| 50.0|    1|
|[2022-05-15 12:00...| 20.0|    3|
|[2022-06-04 12:00...| 16.0|    1|
|[2022-04-20 12:00...| 47.0|    1|
|[2022-04-30 12:00...| 38.0|    1|
|[2022-03-29 00:00...| 19.0|   14|
|[2022-05-28 00:00...| 26.0|    5|
|[2022-04-05 12:00...| 32.0|    1|
|[2022-06-07 00:00...| 16.0|   19|
|[2022-05-08 00:00...| 24.0|    9|
|[2022-04-30 12:00...| 24.0|    7|
|[2022-04-05 12:00...| 49.0|    2|
+--------------------+-----+-----+
only showing top 20 rows



-------------------------------------------
Batch: 70
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-06-09 12:00...| 12.0|    3|
|[2022-04-28 00:00...| 24.0|    7|
|[2022-03-31 12:00...| 23.0|    1|
|[2022-04-08 00:00...| 64.0|    2|
|[2022-05-10 12:00...| 12.0|   12|
|[2022-06-02 00:00...| 25.0|    2|
|[2022-03-31 12:00...| 29.0|    1|
|[2022-04-10 12:00...| 63.0|    1|
|[2022-04-18 00:00...| 50.0|    1|
|[2022-05-15 12:00...| 20.0|    3|
|[2022-06-04 12:00...| 16.0|    1|
|[2022-04-20 12:00...| 47.0|    1|
|[2022-04-30 12:00...| 38.0|    1|
|[2022-03-29 00:00...| 19.0|   14|
|[2022-05-28 00:00...| 26.0|    5|
|[2022-04-05 12:00...| 32.0|    1|
|[2022-06-07 00:00...| 16.0|   19|
|[2022-05-08 00:00...| 24.0|    9|
|[2022-04-30 12:00...| 24.0|    7|
|[2022-04-05 12:00...| 49.0|    2|
+--------------------+-----+-----+
only showing top 20 rows



In [ ]:
query.stop()